# Myelin Mapping Notebook

These workflows process T1 and T2 weighted images to then be used to quantify cortical myelin content per methods described in Gordon et al., 2017 (doi: 10.1016/j.neuron.2017.07.011)

In [1]:
# Import all othe functions/nodes being used
from nipype.pipeline.engine import Workflow, Node, MapNode
from nipype.interfaces.utility import IdentityInterface, Function
from nipype.interfaces.io import SelectFiles, DataSink, DataGrabber, FreeSurferSource
from nipype.interfaces.fsl import FSLCommand, BET, Reorient2Std, FAST, FLIRT
from nipype.interfaces.freesurfer import BBRegister, Binarize, ReconAll, MRIConvert
from nipype.interfaces.ants import N4BiasFieldCorrection
from pandas import read_csv

# Declare study-specific variables
setup='Cat'

if setup=='Jay':
    study_home = '/Users/dendrite/Desktop/Jay'
    raw_data = study_home + '/raw'
elif setup=='Cat':
    study_home = '/data/perlman/moochie/user_data/CamachoCat/5YOP'
    raw_data = '/data/perlman/moochie/study_data/5YOP/MRI_processing'
    temp_fs_dir = '/home/usr/camachoc/Documents/5YOP/freesurfer'
    
output_dir = study_home + '/proc'
workflow_dir = study_home + '/workflows'
fs_dir = study_home + '/freesurfer'

subject_ids = ['3000','4000']

# Set any processing settings
FSLCommand.set_default_output_type('NIFTI_GZ')

200315-14:31:00,494 nipype.utils INFO:
	 No new version available.


In [2]:
# Identity node- select subjects
infosource = Node(IdentityInterface(fields=['subject_id']),
                     name='infosource')
infosource.iterables = ('subject_id', subject_ids)

# Selectfiles node to grab subject T2w data
t2w_templates = {'T2anat': raw_data + '/{subject_id}/ANAT_NIFTIS/t2w_2.nii.gz'}
t2w_source = Node(SelectFiles(t2w_templates), name='t2w_source')

# datasink node
datasink = Node(DataSink(base_directory=output_dir), name='datasink')

## Preprocess T1w data

In [ ]:
# grab raw T1-weighted images and process through freesurfer
t1w_template = {'t1_anats':raw_data + '/%s/ANAT_NIFTIS/t1w_*.nii.gz'}
t1w_source = Node(DataGrabber(base_directory = raw_data, 
                              field_template = t1w_template,
                              template=raw_data + '/%s/ANAT_NIFTIS/t1w_*.nii.gz',
                              sort_filelist=True,
                              in_fields=['subject_id'],
                              outfields=['t1_anats'],
                              template_args={'t1_anats':[['subject_id']]}), 
                  name='t1w_source')

#process through recon-all
reconall = Node(ReconAll(directive='all',flags='-gcut',openmp=4,
                         subjects_dir=temp_fs_dir,use_T2=True), name='reconall')

In [ ]:
fs_proc = Workflow(name='freesurfer_proc_flow')
fs_proc.connect([(infosource, t1w_source, [('subject_id','subject_id')]),
                 (infosource, t2w_source, [('subject_id','subject_id')]),
                 (infosource, reconall, [('subject_id','subject_id')]),
                 (t1w_source, reconall, [('t1_anats','T1_files')]), 
                 (t2w_source, reconall, [('T2anat','T2_file')])
                ])
fs_proc.base_dir = workflow_dir
fs_proc.write_graph(graph2use='flat')
fs_proc.run('MultiProc', plugin_args={'n_procs': 10, 'memory_gb': 20})

## Perform T2 preprocessing and compute T1/T2 ratio
The nodes and workflow below preprocess T2-weighted images to match T1-weighted data processed externally using FreeSurfer v6.  Next, T1/T2 ration is computed on a voxel-wise basis.

In [3]:
#Function to quantify T1/T2 ratio
def quantify_ratio(T2_nifti, T1_nifti):
    from nibabel import load, save, Nifti1Image
    from numpy import divide, zeros
    from os.path import abspath
    
    #import images
    t1w_img = load(T1_nifti)
    t2w_img = load(T2_nifti)
    t1w_data = t1w_img.get_fdata().astype(float)
    t2w_data = t2w_img.get_fdata().astype(float)
    
    #calculate ratio and create new image
    ratio_data = divide(t1w_data, t2w_data)
    ratio_img = Nifti1Image(ratio_data, header=t1w_img.header, affine=t1w_img.affine)
    save(ratio_img, 'ratio_image.nii.gz')
    ratio_file = abspath('ratio_image.nii.gz')
    return(ratio_file)

In [6]:
# FreeSurferSource node to grab processed T1w data from freesurfer
t1w_source = Node(FreeSurferSource(subjects_dir=fs_dir), name='t1w_source')

# skullstrip the T2w using BET (FSL)
t2strip = Node(BET(), name='t2strip')

# N4 bias correction of the T2w data (ANTs)
bias_correct = Node(N4BiasFieldCorrection(),name='bias_correct')

# Reorient T2w to standard
reorient_t2w = Node(Reorient2Std(), name='reorient_t2w')

# Convert T1w from .mgz to niigz
convertt1 = Node(MRIConvert(out_type='niigz',out_file='brain.nii.gz'), name='convertt1')

# Register T2w to T1w using FreeSurfer's Boundary-Based Registration
register = Node(BBRegister(contrast_type='t2', out_fsl_file=True, 
                           subjects_dir=fs_dir), name='register')

apply_registration = Node(FLIRT(apply_xfm=True, out_file='reg_T2.nii.gz'),
                          name='apply_registration')

#Calculate T1/T2 ratio
compute_T1T2_ratio = Node(Function(input_names=['T2_nifti','T1_nifti'],
                                   output_names=['ratio_file'], 
                                   function=quantify_ratio), name='compute_T1T2_ratio')

In [7]:
# Write the workflow
preprocess_t2 = Workflow(name='preprocess_t2')
preprocess_t2.connect([(infosource, t1w_source, [('subject_id','subject_id')]),
                       (t1w_source, convertt1, [('brain','in_file')]),
                       (convertt1, compute_T1T2_ratio, [('out_file','T1_nifti')]),
                       (infosource, t2w_source, [('subject_id','subject_id')]),
                       (infosource, register, [('subject_id','subject_id')]),
                       (t2w_source, t2strip,[('T2anat','in_file')]),
                       (t2strip, reorient_t2w,[('out_file','in_file')]),
                       (reorient_t2w, bias_correct,[('out_file','input_image')]),
                       (bias_correct, register,[('output_image','source_file')]),
                       
                       (bias_correct,apply_registration,[('output_image','in_file')]),
                       (convertt1, apply_registration, [('out_file','reference')]),
                       (register, apply_registration, [('out_fsl_file','in_matrix_file')]),
                       (apply_registration, compute_T1T2_ratio,[('out_file','T2_nifti')]),
                       
                       (compute_T1T2_ratio, datasink, [('ratio_file','t1_t2_ratio')]),
                       (t2strip, datasink,[('out_file','skullstripped_t2w')]),
                       (apply_registration, datasink,[('out_file','registered_t2w')])
                       ])

preprocess_t2.base_dir = workflow_dir
#preprocess_t2.write_graph(graph2use='flat')
preprocess_t2.run('MultiProc', plugin_args={'n_procs': 2, 'memory_gb': 10})

200315-14:31:49,773 nipype.workflow INFO:
	 Workflow preprocess_t2 settings: ['check', 'execution', 'logging', 'monitoring']
200315-14:31:49,795 nipype.workflow INFO:
	 Running in parallel.
200315-14:31:49,798 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 4 jobs ready. Free memory (GB): 10.00/10.00, Free processors: 2/2.
200315-14:31:49,854 nipype.workflow INFO:
	 [Node] Setting-up "preprocess_t2.t2w_source" in "/data/perlman/moochie/user_data/CamachoCat/5YOP/workflows/preprocess_t2/_subject_id_4000/t2w_source".
200315-14:31:49,857 nipype.workflow INFO:
	 [Node] Setting-up "preprocess_t2.t1w_source" in "/data/perlman/moochie/user_data/CamachoCat/5YOP/workflows/preprocess_t2/_subject_id_4000/t1w_source".
200315-14:31:49,867 nipype.workflow INFO:
	 [Node] Running "t2w_source" ("nipype.interfaces.io.SelectFiles")
200315-14:31:49,870 nipype.workflow INFO:
	 [Node] Running "t1w_source" ("nipype.interfaces.io.FreeSurferSource")
200315-14:31:49,885 nipype.workflow INFO:
	 [Node] Fi

<string>:13: RuntimeWarning: divide by zero encountered in true_divide
<string>:13: RuntimeWarning: invalid value encountered in true_divide


200315-14:32:02,533 nipype.workflow INFO:
	 [Node] Finished "preprocess_t2.compute_T1T2_ratio".
200315-14:32:03,814 nipype.workflow INFO:
	 [Job 8] Completed (preprocess_t2.compute_T1T2_ratio).
200315-14:32:03,818 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 2 jobs ready. Free memory (GB): 10.00/10.00, Free processors: 2/2.
200315-14:32:03,865 nipype.workflow INFO:
	 [Node] Setting-up "preprocess_t2.datasink" in "/data/perlman/moochie/user_data/CamachoCat/5YOP/workflows/preprocess_t2/_subject_id_4000/datasink".
200315-14:32:03,865 nipype.workflow INFO:
	 [Node] Setting-up "preprocess_t2.t1w_source" in "/data/perlman/moochie/user_data/CamachoCat/5YOP/workflows/preprocess_t2/_subject_id_3000/t1w_source".
200315-14:32:03,875 nipype.workflow INFO:
	 [Node] Running "t1w_source" ("nipype.interfaces.io.FreeSurferSource")
200315-14:32:03,878 nipype.workflow INFO:
	 [Node] Running "datasink" ("nipype.interfaces.io.DataSink")
200315-14:32:03,893 nipype.workflow INFO:
	 [Node] Finishe

<string>:13: RuntimeWarning: divide by zero encountered in true_divide
<string>:13: RuntimeWarning: invalid value encountered in true_divide


200315-14:32:10,526 nipype.workflow INFO:
	 [Node] Finished "preprocess_t2.compute_T1T2_ratio".
200315-14:32:11,822 nipype.workflow INFO:
	 [Job 18] Completed (preprocess_t2.compute_T1T2_ratio).
200315-14:32:11,827 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 10.00/10.00, Free processors: 2/2.
200315-14:32:11,880 nipype.workflow INFO:
	 [Node] Setting-up "preprocess_t2.datasink" in "/data/perlman/moochie/user_data/CamachoCat/5YOP/workflows/preprocess_t2/_subject_id_3000/datasink".
200315-14:32:11,893 nipype.workflow INFO:
	 [Node] Running "datasink" ("nipype.interfaces.io.DataSink")
200315-14:32:12,41 nipype.workflow INFO:
	 [Node] Finished "preprocess_t2.datasink".
200315-14:32:13,824 nipype.workflow INFO:
	 [Job 19] Completed (preprocess_t2.datasink).
200315-14:32:13,827 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 0 jobs ready. Free memory (GB): 10.00/10.00, Free processors: 2/2.
